In [ ]:
%pip install zeroconf packaging pyttsx3 speechrecognition pyaudio requests

: 

In [ ]:
%pip install --upgrade zeroconf


In [ ]:
%pip uninstall vosk


In [ ]:
import pyttsx3
import speech_recognition as sr
import datetime
import os
import time

import random
import requests  # To send HTTP requests to ESP32
from packaging.version import Version
from zeroconf import Zeroconf, ServiceBrowser

# Initialize the speech engine
engine = pyttsx3.init("sapi5")
voices = engine.getProperty("voices")
engine.setProperty("voice", voices[0].id)

# Path for storing tasks and reminders
TASKS_FILE = "tasks_reminders.txt"

# Path for music directory
MUSIC_DIR = r"C:\Users\hemak\Music"  # Replace with the path to your music folder

# Discover the ESP32 IP address using mDNS (automatically fetch ESP32 IP)
from zeroconf import Zeroconf, ServiceBrowser

class MyListener:
    def __init__(self):
        self.esp32_ip = None

    def add_service(self, zeroconf, type, name):
        info = zeroconf.get_service_info(type, name)
        if info:
            ip = ".".join(map(str, info.addresses[0]))
            print(f"Discovered: {name}, IP: {ip}")
            if "esp32-device" in name:
                self.esp32_ip = ip

def get_esp32_ip():
    zeroconf = Zeroconf()
    listener = MyListener()
    browser = ServiceBrowser(zeroconf, "_http._tcp.local.", listener)
    
    import time
    time.sleep(5)
    zeroconf.close()
    
    return listener.esp32_ip

esp32_ip = get_esp32_ip()
if esp32_ip:
    print(f"Found ESP32 IP: {esp32_ip}")
else:
    print("ESP32 not found.")

# Get the ESP32 IP address dynamically
esp32_ip = get_esp32_ip()
if esp32_ip:
    ESP32_IP = f"http://{esp32_ip}"  # Format the ESP32 IP for HTTP requests
    print(f"ESP32 IP: {ESP32_IP}")
else:
    ESP32_IP = None
    print("ESP32 not found.")

# Load tasks and reminders from file
def load_tasks_reminders():
    tasks = []
    try:
        with open(TASKS_FILE, "r") as file:
            for line in file:
                task_time, task = line.strip().split(" | ")
                tasks.append((datetime.datetime.strptime(task_time, "%Y-%m-%d %H:%M"), task))
    except FileNotFoundError:
        pass
    except ValueError:
        print(f"Skipping invalid line: {line.strip()}")
    return tasks

# Save tasks and reminders to file
def save_tasks_reminders(tasks):
    with open(TASKS_FILE, "w") as file:
        for task_time, task in tasks:
            file.write(f"{task_time.strftime('%Y-%m-%d %H:%M')} | {task}\n")

# Initialize tasks and reminders
scheduled_tasks = load_tasks_reminders()

# Function to convert text to speech
def speak(audio):
    engine.say(audio)
    print(f"Speaking: {audio}")
    engine.runAndWait()

# Greet the user based on the time of day
def greet_user():
    hour = int(datetime.datetime.now().hour)
    if hour >= 0 and hour < 12:
        speak("Good Morning Chief!")
    elif hour >= 12 and hour < 18:
        speak("Good Afternoon Chief!")
    else:
        speak("Good Evening Chief!")

    speak("I am Jarvis, your personal assistant. I can tell you the time, date, set reminders, play songs, control lights, open applications, and more. How can I assist you today?")

# Function to capture voice command and convert it to text
def takecommand(timeout=5):
    r = sr.Recognizer()

    try:
        import pyaudio
        if Version(pyaudio.__version__) < Version("0.2.11"):
            raise AttributeError(f"PyAudio 0.2.11 or later is required (found version {pyaudio.__version__})")
    except ImportError:
        raise AttributeError("PyAudio is not installed. Please install it using 'pip install pyaudio'.")

    with sr.Microphone() as source:
        print("Listening...")
        r.pause_threshold = 1
        r.energy_threshold = 300
        r.adjust_for_ambient_noise(source, duration=1)
        try:
            audio = r.listen(source, timeout=timeout, phrase_time_limit=5)
        except sr.WaitTimeoutError:
            return "none"

    try:
        print("Recognizing...")
        query = r.recognize_google(audio, language="en-in")
        print(f"User said: {query}")
    except sr.UnknownValueError:
        return "none"
    except sr.RequestError as e:
        print(f"API error: {e}")
        return "none"
    except Exception as e:
        print(f"Error: {e}")
        return "none"

    return query

# Function to send a command to ESP32 to turn on/off a light
def control_light(light_number, action):
    try:
        # Validate light_number and action
        if light_number not in ["1", "2", "3"]:
            speak("Invalid light number. Please specify light 1, 2, or 3.")
            return
        
        if action not in ["on", "off"]:
            speak("Invalid action. Please specify 'on' or 'off'.")
            return
        
        if ESP32_IP:  # Ensure that ESP32 IP is available
            url = f"{ESP32_IP}/light{light_number}/{action}"
            response = requests.get(url)
            response.raise_for_status()  # Raise an error for HTTP codes 4xx/5xx
            
            if response.status_code == 200:
                speak(f"Light {light_number} is now {action.upper()}.")
            else:
                speak(f"Failed to {action} light {light_number}.")
        else:
            speak("Couldn't reach the ESP32. ESP32 IP is not available.")
    except requests.RequestException as e:
        speak(f"Couldn't reach the ESP32 for light {light_number}. Error: {e}")
        print(f"Request error: {e}")
    except Exception as e:
        speak(f"An unexpected error occurred while controlling light {light_number}.")
        print(f"Error: {e}")

# Function to set a reminder
def set_reminder(reminder_time, reminder_message):
    try:
        reminder_time = datetime.datetime.strptime(reminder_time, "%H:%M")
        reminder_time = reminder_time.replace(year=datetime.datetime.now().year, 
                                              month=datetime.datetime.now().month, 
                                              day=datetime.datetime.now().day)
        scheduled_tasks.append((reminder_time, reminder_message))
        save_tasks_reminders(scheduled_tasks)
        speak(f"Reminder set for {reminder_time.strftime('%H:%M')}. I'll remind you.")
    except ValueError:
        speak("Sorry, I couldn't understand the time format. Please use the 24-hour format, like '15:30'.")

# Function to check and execute scheduled tasks
def check_scheduled_tasks():
    current_time = datetime.datetime.now()
    for task_time, task in scheduled_tasks:
        if task_time <= current_time:
            speak(f"Reminder: {task}")
            scheduled_tasks.remove((task_time, task))
            save_tasks_reminders(scheduled_tasks)

# Function to play a random song
def play_song():
    try:
        songs = os.listdir(MUSIC_DIR)
        song = random.choice(songs)
        speak(f"Playing {song}")
        os.startfile(os.path.join(MUSIC_DIR, song))  # Use default media player
    except Exception as e:
        speak("Sorry, I couldn't play the song.")
        print(f"Error: {e}")

# Function to respond to recognized commands
def respond(query):
    query = query.lower()

    # Map words to numbers
    number_words = {"one": "1", "two": "2", "three": "3"}
    
    # Convert number words to numbers
    for word, number in number_words.items():
        if word in query:
            query = query.replace(word, number)

    if "time" in query:
        current_time = datetime.datetime.now().strftime("%H:%M:%S")
        speak(f"The time is {current_time}")
    elif "date" in query:
        current_date = datetime.datetime.now().strftime("%A, %B %d, %Y")
        speak(f"Today's date is {current_date}")
    elif "name" in query:
        speak("My name is Jarvis, your personal assistant.")
    elif "do you know me" in query:
        speak("Yes, you are HemaKoti Bonthada, Chief of me.")
    elif "how are you" in query:
        speak("I'm doing great, thank you for asking!")
    elif "who are you" in query:
        speak("I am an AI assistant created to help you with your tasks.")
    elif "what can you do" in query:
        speak("I can tell you the time, date, set reminders, play music, control lights, and more!")
    elif "remind me" in query or "set a reminder" in query or "schedule" in query:
        try:
            if "at" in query:
                parts = query.split(" at ")
                reminder_time = parts[1].strip().split()[0]
                reminder_message = parts[0].replace("remind me to", "").strip()
                set_reminder(reminder_time, reminder_message)
            else:
                speak("Please specify the time for the reminder.")
        except Exception as e:
            speak("I couldn't set the reminder. Please try again.")
            print(f"Error in reminder: {e}")
    elif "play" in query and "song" in query:
        play_song()
    elif "turn on light" in query:
        control_light(query.split("light")[1].strip(), "on")
    elif "turn off light" in query:
        control_light(query.split("light")[1].strip(), "off")
    else:
        speak("I'm sorry, I didn't understand that command.")

# Main function to listen for the wake word
def listen_for_jarvis():
    greet_user()
    while True:
        print("Say 'Jarvis' to wake me up.")
        query = takecommand(timeout=5).lower()
        if "jarvis" in query:
            speak("I'm listening, Chief.")
            query = takecommand(timeout=10)
            if query == "none":
                speak("I couldn't hear you clearly.")
            else:
                respond(query)

if __name__ == "__main__":
    listen_for_jarvis()


In [ ]:
import pyttsx3
import speech_recognition as sr
import datetime
import os
import time
import random
import requests
from packaging.version import Version
from zeroconf import Zeroconf, ServiceBrowser

# Initialize the speech engine
engine = pyttsx3.init("sapi5")
voices = engine.getProperty("voices")
engine.setProperty("voice", voices[1].id)  # Use female voice

# Path for storing tasks and reminders
TASKS_FILE = "tasks_reminders.txt"

# Path for music directory
MUSIC_DIR = r"C:\Users\hemak\Music"  # Replace with the path to your music folder

# Discover the ESP32 IP address using mDNS (automatically fetch ESP32 IP)
class MyListener:
    def __init__(self):
        self.esp32_ip = None

    def add_service(self, zeroconf, type, name):
        info = zeroconf.get_service_info(type, name)
        if info:
            ip = ".".join(map(str, info.addresses[0]))
            print(f"Discovered: {name}, IP: {ip}")
            if "esp32-device" in name:
                self.esp32_ip = ip

def get_esp32_ip():
    zeroconf = Zeroconf()
    listener = MyListener()
    browser = ServiceBrowser(zeroconf, "_http._tcp.local.", listener)
    
    import time
    time.sleep(5)
    zeroconf.close()
    
    return listener.esp32_ip

esp32_ip = get_esp32_ip()
if esp32_ip:
    print(f"Found ESP32 IP: {esp32_ip}")
else:
    print("ESP32 not found.")

# Load tasks and reminders from file
def load_tasks_reminders():
    tasks = []
    try:
        with open(TASKS_FILE, "r") as file:
            for line in file:
                task_time, task = line.strip().split(" | ")
                tasks.append((datetime.datetime.strptime(task_time, "%Y-%m-%d %H:%M"), task))
    except FileNotFoundError:
        pass
    except ValueError:
        print(f"Skipping invalid line: {line.strip()}")
    return tasks

# Save tasks and reminders to file
def save_tasks_reminders(tasks):
    with open(TASKS_FILE, "w") as file:
        for task_time, task in tasks:
            file.write(f"{task_time.strftime('%Y-%m-%d %H:%M')} | {task}\n")

# Initialize tasks and reminders
scheduled_tasks = load_tasks_reminders()

# Function to convert text to speech
def speak(audio):
    engine.say(audio)
    print(f"Speaking: {audio}")
    engine.runAndWait()

# Greet the user based on the time of day
def greet_user():
    hour = int(datetime.datetime.now().hour)
    if hour >= 0 and hour < 12:
        speak("Good Morning Chief!")
    elif hour >= 12 and hour < 18:
        speak("Good Afternoon Chief!")
    else:
        speak("Good Evening Chief!")

    speak("I am Bujee, your personal assistant. How can I assist you today?")

# Function to capture voice command and convert it to text
def takecommand(timeout=5):
    r = sr.Recognizer()

    try:
        import pyaudio
        if Version(pyaudio.__version__) < Version("0.2.11"):
            raise AttributeError(f"PyAudio 0.2.11 or later is required (found version {pyaudio.__version__})")
    except ImportError:
        raise AttributeError("PyAudio is not installed. Please install it using 'pip install pyaudio'.")

    with sr.Microphone() as source:
        print("Listening...")
        r.pause_threshold = 1
        r.energy_threshold = 300
        r.adjust_for_ambient_noise(source, duration=1)
        try:
            audio = r.listen(source, timeout=timeout, phrase_time_limit=5)
        except sr.WaitTimeoutError:
            return "none"

    try:
        print("Recognizing...")
        query = r.recognize_google(audio, language="en-IN")
        print(f"User said: {query}")
    except sr.UnknownValueError:
        return "none"
    except sr.RequestError as e:
        print(f"API error: {e}")
        return "none"
    except Exception as e:
        print(f"Error: {e}")
        return "none"

    return query

# Function to send a command to ESP32 to turn on/off a light
def control_light(light_number, action):
    try:
        if light_number not in ["1", "2", "3"]:
            speak("Invalid light number. Please specify light 1, 2, or 3.")
            return
        if action not in ["on", "off"]:
            speak("Invalid action. Please specify 'on' or 'off'.")
            return
        if ESP32_IP:
            url = f"{ESP32_IP}/light{light_number}/{action}"
            response = requests.get(url)
            response.raise_for_status()
            if response.status_code == 200:
                speak(f"Light {light_number} is now {action.upper()}.")
            else:
                speak(f"Failed to {action} light {light_number}.")
        else:
            speak("Couldn't reach the ESP32. ESP32 IP is not available.")
    except requests.RequestException as e:
        speak(f"Couldn't reach the ESP32 for light {light_number}. Error: {e}")
        print(f"Request error: {e}")
    except Exception as e:
        speak(f"An unexpected error occurred while controlling light {light_number}.")
        print(f"Error: {e}")

# Function to set a reminder
def set_reminder(reminder_time, reminder_message):
    try:
        reminder_time = datetime.datetime.strptime(reminder_time, "%H:%M")
        reminder_time = reminder_time.replace(year=datetime.datetime.now().year, 
                                              month=datetime.datetime.now().month, 
                                              day=datetime.datetime.now().day)
        scheduled_tasks.append((reminder_time, reminder_message))
        save_tasks_reminders(scheduled_tasks)
        speak(f"Reminder set for {reminder_time.strftime('%H:%M')}. I'll remind you.")
    except ValueError:
        speak("Sorry, I couldn't understand the time format. Please use the 24-hour format, like '15:30'.")

# Function to check and execute scheduled tasks
def check_scheduled_tasks():
    current_time = datetime.datetime.now()
    for task_time, task in scheduled_tasks:
        if task_time <= current_time:
            speak(f"Reminder: {task}")
            scheduled_tasks.remove((task_time, task))
            save_tasks_reminders(scheduled_tasks)

# Function to play a random song
def play_song():
    try:
        songs = os.listdir(MUSIC_DIR)
        song = random.choice(songs)
        speak(f"Playing {song}")
        os.startfile(os.path.join(MUSIC_DIR, song))  # Use default media player
    except Exception as e:
        speak("Sorry, I couldn't play the song.")
        print(f"Error: {e}")

# Function to respond to recognized commands
def respond(query):
    query = query.lower()

    number_words = {"one": "1", "two": "2", "three": "3"}
    for word, number in number_words.items():
        if word in query:
            query = query.replace(word, number)

    if "time" in query:
        current_time = datetime.datetime.now().strftime("%H:%M:%S")
        speak(f"The time is {current_time}")
    elif "date" in query:
        current_date = datetime.datetime.now().strftime("%A, %B %d, %Y")
        speak(f"Today's date is {current_date}")
    elif "name" in query:
        speak("My name is Bujee, your personal assistant.")
    elif "do you know me" in query:
        speak("Yes, you are HemaKoti Bonthada, Chief of me.")
    elif "how are you" in query:
        speak("I'm doing great, thank you for asking!")
    elif "who are you" in query:
        speak("I am an AI assistant created to help you with your tasks.")
    elif "what can you do" in query:
        speak("I can tell you the time, date, set reminders, play music, control lights, and more!")
    elif "remind me" in query or "set a reminder" in query or "schedule" in query:
        try:
            if "at" in query:
                parts = query.split(" at ")
                reminder_time = parts[1].strip().split()[0]
                reminder_message = parts[0].replace("remind me to", "").strip()
                set_reminder(reminder_time, reminder_message)
            else:
                speak("Please specify the time for the reminder.")
        except Exception as e:
            speak("I couldn't set the reminder. Please try again.")
            print(f"Error in reminder: {e}")
    elif "play" in query and "song" in query:
        play_song()
    elif "turn on light" in query:
        control_light(query.split("light")[1].strip(), "on")
    elif "turn off light" in query:
        control_light(query.split("light")[1].strip(), "off")
    else:
        speak("I'm sorry, I didn't understand that command.")

# Main function to listen for the wake word
def listen_for_bujji():
    greet_user()
    while True:
        print("Say 'Bujji' to wake me up.")
        query = takecommand(timeout=5).lower()
        if "bujji" in query:
            speak("I'm listening, Chief.")
            query = takecommand(timeout=10)
            if query == "none":
                speak("I couldn't hear you clearly.")
            else:
                respond(query)

if __name__ == "__main__":
    listen_for_bujji()


In [ ]:
import subprocess
import re

def get_esp32_ip(mac_address_prefix):
    try:
        # Run arp command to get the ARP table
        output = subprocess.check_output(['arp', '-a'], text=True)

        # Regex pattern to match MAC address and IP address
        pattern = re.compile(r'\((\d+\.\d+\.\d+\.\d+)\)\s+([\w-:]+)')

        # Iterate through the matched results
        for line in output.splitlines():
            match = pattern.search(line)
            if match:
                ip_address, mac_address = match.groups()
                if mac_address.startswith(mac_address_prefix):
                    return ip_address

    except subprocess.CalledProcessError as e:
        print(f"Error running arp command: {e}")

    return None

if __name__ == "__main__":
    mac_prefix = 'xx-xx-xx'  # Replace with the MAC address prefix of your ESP32
    esp32_ip = get_esp32_ip(mac_prefix)
    if esp32_ip:
        print(f"ESP32 IP Address: {esp32_ip}")
    else:
        print("ESP32 not found.")
